In [108]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code','') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True

time.sleep(60)

for i in range(5000,5570):
    
    # initialize values
    scholar_id = data.iat[i, 3]
    citation = data.iat[i, 4]
    h_index = data.iat[i, 5]

    if (data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i, 3] == "#NAME?" or isnan(data.iat[i, 4]) or isnan(data.iat[i, 5])):
        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] + " " + data.iat[i, 1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
            
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")
                if elements_list: 
                    for element in elements_list:  # looping through the number of professors in the link
                        try:
                            element.find_element(By.TAG_NAME, "b")
                            element.click()
                            professorExists = True
                            break
                        except NoSuchElementException:
                            professorExists = False
                            continue  # in the case that the element is not found
                    # give time for html / css / js to load
                    time.sleep(random() * 5)
                else: 
                    professorExists = False 

        # retrieve citation, h-index, and scholar_id for that prof
        if professorExists:
            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = int(driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text)
            h_index = int(driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text)
            time.sleep(random() * 5)
        else:
            scholar_id = None
            citation = None
            h_index = None
    
    print(i)
    professor = { 
        "Name": data.iat[i, 0],
        "affiliation": data.iat[i, 1],
        "homepage": data.iat[i, 2],
        "scholar-id": scholar_id,
        "citations": citation,
        "h-index": h_index
    }
    print(professor)
    professors.append(professor)


df = pd.DataFrame(professors)
df.to_csv(os.getcwd().replace('code','') +"data/data1.csv", index = False)


[WDM] - ====== WebDriver manager ======


2022-07-30 16:27:16,391 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-30 16:27:16,482 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

5428
{'Name': 'Wouter Joosen', 'affiliation': 'KU Leuven', 'homepage': 'https://distrinet.cs.kuleuven.be/people/wouter', 'scholar-id': None, 'citations': None, 'h-index': None}
5429
{'Name': 'Tony Wauters', 'affiliation': 'KU Leuven', 'homepage': 'https://people.cs.kuleuven.be/~tony.wauters', 'scholar-id': 'moTbWMUAAAAJ', 'citations': 911.0, 'h-index': 17.0}
5430
{'Name': 'Tom Schrijvers', 'affiliation': 'KU Leuven', 'homepage': 'https://people.cs.kuleuven.be/~tom.schrijvers', 'scholar-id': 'JVXShtsAAAAJ', 'citations': 3761.0, 'h-index': 30.0}
5431
{'Name': 'Stefan Vandewalle', 'affiliation': 'KU Leuven', 'homepage': 'https://people.cs.kuleuven.be/~stefan.vandewalle', 'scholar-id': 'Y-en1DsAAAAJ', 'citations': 5172.0, 'h-index': 40.0}
5432
{'Name': 'Vincent Rijmen', 'affiliation': 'KU Leuven', 'homepage': 'https://www.esat.kuleuven.be/cosic/people/vincent-rijmen', 'scholar-id': 'zBQxZrcAAAAJ', 'citations': 22694.0, 'h-index': 59.0}
5433
{'Name': 'Greet Vanden Berghe', 'affiliation': 'K

5471
{'Name': 'Franziska Boehm', 'affiliation': 'Karlsruhe Institute of Technology', 'homepage': 'https://www.zar.kit.edu/492.php?ID=237', 'scholar-id': None, 'citations': None, 'h-index': None}
5472
{'Name': 'Jürgen Beyerer', 'affiliation': 'Karlsruhe Institute of Technology', 'homepage': 'http://ies.anthropomatik.kit.edu/mitarbeiter.php?person=beyerer', 'scholar-id': '-BCKzg0AAAAJ', 'citations': 5346, 'h-index': 32}
5473
{'Name': 'Björn Hein', 'affiliation': 'Karlsruhe Institute of Technology', 'homepage': 'http://rob.ipr.kit.edu/mitarbeiter_106.php', 'scholar-id': 'RqTDkH4AAAAJ', 'citations': 829, 'h-index': 15}
5474
{'Name': 'Uwe D. Hanebeck', 'affiliation': 'Karlsruhe Institute of Technology', 'homepage': 'https://isas.iar.kit.edu/de/Mitarbeiter_Hanebeck.php', 'scholar-id': 'q-f3ITAAAAAJ', 'citations': 9266.0, 'h-index': 43.0}
5475
{'Name': 'Gregor Snelting', 'affiliation': 'Karlsruhe Institute of Technology', 'homepage': 'https://pp.info.uni-karlsruhe.de/personhp/gregor_snelting.

5516
{'Name': 'Cheng Chang Lu', 'affiliation': 'Kent State University', 'homepage': 'http://www.cs.kent.edu/~lucc', 'scholar-id': None, 'citations': None, 'h-index': None}
5517
{'Name': 'Jungyong Kim', 'affiliation': 'Kent State University', 'homepage': 'https://www.kent.edu/cs/jungyoon-kim', 'scholar-id': 'z2BCKT4AAAAJ', 'citations': 547.0, 'h-index': 9.0}
5518
{'Name': 'Gokarna Sharma', 'affiliation': 'Kent State University', 'homepage': 'http://www.cs.kent.edu/~sharma', 'scholar-id': 'PnmtA3wAAAAJ', 'citations': 863.0, 'h-index': 15.0}
5519
{'Name': 'Kwangtaek Kim', 'affiliation': 'Kent State University', 'homepage': 'https://www.kent.edu/cs/kwangtaek-kim', 'scholar-id': None, 'citations': None, 'h-index': None}
5520
{'Name': 'Kambiz Ghazinour', 'affiliation': 'Kent State University', 'homepage': 'https://www.kent.edu/cs/profile/kambiz-ghazinour', 'scholar-id': 'S5Dabo0AAAAJ', 'citations': 571.0, 'h-index': 12.0}
5521
{'Name': 'Hassan Peyravi', 'affiliation': 'Kent State University'

5560
{'Name': 'Amin Y. Noaman', 'affiliation': 'King Abdulaziz University', 'homepage': 'http://anoaman.kau.edu.sa/CVEn.aspx?Site_ID=0002639&Lng=EN', 'scholar-id': None, 'citations': None, 'h-index': None}
5561
{'Name': 'Ahmad Alzahrani', 'affiliation': 'King Abdulaziz University', 'homepage': 'http://asalzahrani.kau.edu.sa/Default.aspx?Site_ID=0005199&lng=EN', 'scholar-id': '10GPVtsAAAAJ', 'citations': 227.0, 'h-index': 8.0}
5562
{'Name': 'Areej Malibari', 'affiliation': 'King Abdulaziz University', 'homepage': 'http://aamalibari1.kau.edu.sa/CVEn.aspx?Site_ID=0004507&Lng=EN', 'scholar-id': 'NadtXs0AAAAJ', 'citations': 504, 'h-index': 11}
5563
{'Name': 'Imtiaz Hussain Khan', 'affiliation': 'King Abdulaziz University', 'homepage': 'http://www.kau.edu.sa/Contacts.aspx?Site_ID=61117&Lng=EN&Cont=19678', 'scholar-id': 'RDj7uSYAAAAJ', 'citations': 240.0, 'h-index': 9.0}
5564
{'Name': 'Abdullah Basuhail', 'affiliation': 'King Abdulaziz University', 'homepage': 'http://abasuhail.kau.edu.sa/Def

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='gs_res_ccl_mid']/div[1]"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x00000001063e8ef9 chromedriver + 4480761
1   chromedriver                        0x00000001063745d3 chromedriver + 4003283
2   chromedriver                        0x0000000106007338 chromedriver + 410424
3   chromedriver                        0x000000010603d5bd chromedriver + 632253
4   chromedriver                        0x000000010603d841 chromedriver + 632897
5   chromedriver                        0x000000010606f7d4 chromedriver + 837588
6   chromedriver                        0x000000010605aa8d chromedriver + 752269
7   chromedriver                        0x000000010606d4f1 chromedriver + 828657
8   chromedriver                        0x000000010605a953 chromedriver + 751955
9   chromedriver                        0x0000000106030cd5 chromedriver + 58

In [97]:
professors

[{'Name': 'Qing Yang',
  'affiliation': 'Montana State University',
  'homepage': 'https://www.cs.montana.edu/yang',
  'scholar-id': 'FIMxNL0AAAAJ',
  'citations': 3022,
  'h-index': 30},
 {'Name': 'Upulee Kanewala',
  'affiliation': 'Montana State University',
  'homepage': 'https://www.cs.montana.edu/directory/1812043/upulee-kanewala',
  'scholar-id': 'WIG-usAAAAAJ',
  'citations': 553.0,
  'h-index': 11.0},
 {'Name': 'Binhai Zhu',
  'affiliation': 'Montana State University',
  'homepage': 'https://www.cs.montana.edu/bhz',
  'scholar-id': 'BAzWR-UAAAAJ',
  'citations': 2894,
  'h-index': 27},
 {'Name': 'John Paxton',
  'affiliation': 'Montana State University',
  'homepage': 'https://www.cs.montana.edu/directory/1524459/john-paxton',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Brendan Mumey',
  'affiliation': 'Montana State University',
  'homepage': 'https://www.cs.montana.edu/directory/1524423/brendan-mumey',
  'scholar-id': 'gw5eQs8AAAAJ',
  'citation

In [111]:
df = pd.DataFrame(professors)
df.to_csv(os.getcwd().replace('code','') +"data/data1.csv", index = False)

In [112]:
data.iloc[data['name']== 'Eatai Roth']

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [31]:
big_data = pd.DataFrame()
for i in range(5):
    temp_data = pd.read_csv(os.getcwd().replace('code','') + f'/data/data{i+1}.csv')
    big_data = pd.concat([big_data,temp_data])
big_data.to_csv(os.getcwd().replace('code','') + '/data/cleaned_prof_data_5000_8000.csv', index = False)

In [51]:
data1 = pd.read_csv(os.getcwd().replace('code','') + "/data/replaced_data_8000_20650.csv")
data2 = pd.read_csv(os.getcwd().replace('code','') + "/data/cleaned_prof_data_5000_8000.csv")

In [52]:
data_3 = pd.concat([data1,data2]).reset_index(drop=True)
data_3 = data_3.drop(['Unnamed: 0'], axis = 1)
data_3.to_csv(os.getcwd().replace('code','') + "/data/replaced_data_5000_20650.csv", index = False)

In [53]:
data4 = pd.read_csv(os.getcwd().replace('code','') + "/data/replaced_data_8000_20650.csv")



In [56]:
data4.loc[pd.isna(data['h-index']) & ~pd.isna(data4['h-index'])]

,Name,affiliation,homepage,scholar-id,citations,h-index


In [62]:
data4.loc[~pd.isna(data['scholar_id']) & pd.isna(data4['scholar-id'])].iloc[456]

Name                                   Franz Puntigam
affiliation                                   TU Wien
homepage       http://www.complang.tuwien.ac.at/franz
scholar-id                                        NaN
citations                                         NaN
h-index                                           NaN
Name: 2592, dtype: object